In [15]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [16]:
from utils import Data

data_csv = Data(
    fGEP_SGA = 'data/CITRUS_GEP_SGAseparated.csv',
    fgene_tf_SGA = 'data/CITRUS_gene_tf_SGAseparated.csv',
    fcancerType_SGA = 'data/CITRUS_canType_SGAseparated.csv',
    fSGA_SGA = 'data/CITRUS_SGA_SGAseparated.csv',
)

In [180]:
def shared_tfs(gene_str):
    genes = gene_str.split(',')
    genes = set([g.rstrip().lstrip() for g in genes])
    genes = genes.intersection(set(data_csv.tf))
    return genes if len(genes) >0 else None

In [38]:
pathways2genes = pd.read_csv('/ihome/hosmanbeyoglu/kor11/tools/CITRUS/GenNet/resources/pathways/CPDB_pathways_genes.tab', '\t')[:-1]
kegg_pathways = pd.read_csv('/ihome/hosmanbeyoglu/kor11/tools/CITRUS/GenNet/resources/pathways/pathway_overview_KEGG.csv')
pathways2genes = pathways2genes[pathways2genes.source == 'KEGG']
pathways2genes.pathway = pathways2genes.pathway.str[:-len('- Homo sapiens (human)')]
pathways2genes.pathway = [i.rstrip().lstrip() for i in pathways2genes.pathway]
kegg_pathways.local = [i.rstrip().lstrip() for i in kegg_pathways.local]
pathways2genes['external_id'] = pathways2genes['external_id'].str[len('path:hsa'):].astype(np.int64)

In [183]:
pathways2genes['tf'] = pathways2genes['hgnc_symbol_ids'].apply(shared_tfs)

In [37]:
kegg_pathways

,Unnamed: 0,global,mid,hsaid,local,mid_id,global_id,local_id
0,150,Cellular Processes,Cell growth and death,4110,Cell cycle,7,0,132
1,151,Cellular Processes,Cell growth and death,4114,Oocyte meiosis,7,0,133
2,156,Cellular Processes,Cell growth and death,4115,p53 signaling pathway,7,0,134
3,152,Cellular Processes,Cell growth and death,4210,Apoptosis,7,0,149
4,153,Cellular Processes,Cell growth and death,4215,Apoptosis - multiple species,7,0,152
...,...,...,...,...,...,...,...,...
325,229,Organismal Systems,Nervous system,4730,Long-term depression,32,5,210
326,234,Organismal Systems,Sensory system,4740,Olfactory transduction,36,5,211
327,235,Organismal Systems,Sensory system,4742,Taste transduction,36,5,212
328,233,Organismal Systems,Sensory system,4744,Phototransduction,36,5,213


In [1]:
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly

""


In [11]:
sources[:10]

[0, 0, 0, 1, 1, 1, 2, 2, 2, 5]

In [12]:
targets[:10]

[1, 2, 3, 4, 5, 6, 5, 6, 4, 7]

In [13]:
labels[:10]

['install',
 'signup',
 'purchase',
 'reopen',
 'reopen',
 'purchase',
 'signup',
 'purchase',
 'reopen',
 'signup']

In [108]:
# (pathways2genes.loc[pathways2genes.hgnc_symbol_ids.apply(
#     lambda x: shared_tfs(x)).sort_values(ascending=False)
#     # .replace(0, np.nan)
#     .dropna().index]
#     .join(kegg_pathways, lsuffix='pathway', rsuffix='local', how='inner')
# )[['pathway', 'global', 'local']]

In [131]:
kegg_pathways['mid_id'] += 6

In [157]:
kegg_pathways['local_id'] += 44

In [221]:
kegg_pathways

,Unnamed: 0,global,mid,hsaid,local,mid_id,global_id,local_id,count
0,150,Cellular Processes,Cell growth and death,4110,Cell cycle,13,0,176,1
1,151,Cellular Processes,Cell growth and death,4114,Oocyte meiosis,13,0,177,1
2,156,Cellular Processes,Cell growth and death,4115,p53 signaling pathway,13,0,178,1
3,152,Cellular Processes,Cell growth and death,4210,Apoptosis,13,0,193,1
4,153,Cellular Processes,Cell growth and death,4215,Apoptosis - multiple species,13,0,196,1
...,...,...,...,...,...,...,...,...,...
325,229,Organismal Systems,Nervous system,4730,Long-term depression,38,5,254,1
326,234,Organismal Systems,Sensory system,4740,Olfactory transduction,42,5,255,1
327,235,Organismal Systems,Sensory system,4742,Taste transduction,42,5,256,1
328,233,Organismal Systems,Sensory system,4744,Phototransduction,42,5,257,1


In [229]:
full_kegg = kegg_pathways.copy()

In [158]:
kegg_pathways['count'] = 1

In [252]:
sources2 = []
targets2 = []

# kegg_pathways = full_kegg.join(pathways2genes, lsuffix='local', rsuffix='pathway', how='inner').dropna()
kegg_pathways = full_kegg

labels2 = kegg_pathways[['global', 'global_id']].drop_duplicates().sort_values(by='global_id')['global'].values.tolist() + \
        kegg_pathways[['mid', 'mid_id']].drop_duplicates().sort_values(by='mid_id')['mid'].values.tolist() + \
        kegg_pathways[['local', 'local_id']].drop_duplicates().sort_values(by='local_id')['local'].values.tolist()

values2 = []

for l, global_id, mid_id, count in kegg_pathways[['global', 'global_id', 'mid_id', 'count']].groupby(['global', 'global_id', 'mid_id'])['count'].count().reset_index().values:
    sources2.append(global_id)
    targets2.append(mid_id)
    values2.append(count)
    
    
for l, global_id, mid_id, count in kegg_pathways[['mid', 'mid_id', 'local_id', 'count']].groupby(['mid', 'mid_id', 'local_id'])['count'].count().reset_index().values:
    sources2.append(global_id)
    targets2.append(mid_id)
    values2.append(count)

In [256]:
# fig = go.Figure(data=[go.Sankey(
#     node=dict(
#         thickness=20,  # default is 20
#         line=dict(color="black", width=0.9),
#         label=labels2
#     ),
#     link=dict(
#         source=sources2,
#         target=targets2,
#         value=values2,
#         # label=time_to_next[:len(sources2)],
#         # hovertemplate='%{value} unique users went from %{source.label} to %{target.label}.<br />' +
#         # '<br />It took them %{label} in average.<extra></extra>',
#     ))])

# fig.update_layout(autosize=False, width=1400, height=6000, title_text="Pathways",
#                   font=dict(size=15), plot_bgcolor='white')


# fig.show()

In [264]:
kegg_pathways[['global', 'mid', 'local']].groupby(['global', 'mid', 'local']).count()

Empty DataFrame
Columns: []
Index: [(Cellular Processes, Cell growth and death, Apoptosis), (Cellular Processes, Cell growth and death, Apoptosis - multiple species), (Cellular Processes, Cell growth and death, Cell cycle), (Cellular Processes, Cell growth and death, Cellular senescence), (Cellular Processes, Cell growth and death, Ferroptosis), (Cellular Processes, Cell growth and death, Necroptosis), (Cellular Processes, Cell growth and death, Oocyte meiosis), (Cellular Processes, Cell growth and death, p53 signaling pathway), (Cellular Processes, Cell motility, Regulation of actin cytoskeleton), (Cellular Processes, Cellular community - eukaryotes, Adherens junction), (Cellular Processes, Cellular community - eukaryotes, Focal adhesion), (Cellular Processes, Cellular community - eukaryotes, Gap junction), (Cellular Processes, Cellular community - eukaryotes, Signaling pathways regulating pluripotency of stem cells), (Cellular Processes, Cellular community - eukaryotes, Tight junction), (Cellular Processes, Transport and catabolism, Autophagy - animal), (Cellular Processes, Transport and catabolism, Autophagy - other), (Cellular Processes, Transport and catabolism, Endocytosis), (Cellular Processes, Transport and catabolism, Lysosome), (Cellular Processes, Transport and catabolism, Mitophagy - animal), (Cellular Processes, Transport and catabolism, Peroxisome), (Cellular Processes, Transport and catabolism, Phagosome), (Environmental Information Processing, Membrane transport, ABC transporters), (Environmental Information Processing, Signal transduction, AMPK signaling pathway), (Environmental Information Processing, Signal transduction, Apelin signaling pathway), (Environmental Information Processing, Signal transduction, Calcium signaling pathway), (Environmental Information Processing, Signal transduction, ErbB signaling pathway), (Environmental Information Processing, Signal transduction, FoxO signaling pathway), (Environmental Information Processing, Signal transduction, HIF-1 signaling pathway), (Environmental Information Processing, Signal transduction, Hedgehog signaling pathway), (Environmental Information Processing, Signal transduction, Hippo signaling pathway), (Environmental Information Processing, Signal transduction, Hippo signaling pathway - multiple species), (Environmental Information Processing, Signal transduction, Jak-STAT signaling pathway), (Environmental Information Processing, Signal transduction, MAPK signaling pathway), (Environmental Information Processing, Signal transduction, NF-kappa B signaling pathway), (Environmental Information Processing, Signal transduction, Notch signaling pathway), (Environmental Information Processing, Signal transduction, PI3K-Akt signaling pathway), (Environmental Information Processing, Signal transduction, Phosphatidylinositol signaling system), (Environmental Information Processing, Signal transduction, Phospholipase D signaling pathway), (Environmental Information Processing, Signal transduction, Rap1 signaling pathway), (Environmental Information Processing, Signal transduction, Ras signaling pathway), (Environmental Information Processing, Signal transduction, Sphingolipid signaling pathway), (Environmental Information Processing, Signal transduction, TGF-beta signaling pathway), (Environmental Information Processing, Signal transduction, TNF signaling pathway), (Environmental Information Processing, Signal transduction, VEGF signaling pathway), (Environmental Information Processing, Signal transduction, Wnt signaling pathway), (Environmental Information Processing, Signal transduction, cAMP signaling pathway), (Environmental Information Processing, Signal transduction, cGMP-PKG signaling pathway), (Environmental Information Processing, Signal transduction, mTOR signaling pathway), (Environmental Information Processing, Signaling molecules and interaction, Cell adhesion molecules (CAMs)), (Environmental Information Processing, Signaling molecules and interactio

In [275]:
global2mid = pd.DataFrame(index=kegg_pathways['global'].unique(), columns=kegg_pathways['mid'].unique()).fillna(0)
mid2local = pd.DataFrame(index=kegg_pathways['mid'].unique(), columns=kegg_pathways['local'].unique()).fillna(0)
local2tf = pd.DataFrame(index=kegg_pathways['local'].unique(), columns=data_csv.tf).fillna(0)
global2mid.shape, mid2local.shape, local2tf.shape

((6, 44), (44, 330), (330, 320))

In [279]:
for from_, to_ in kegg_pathways[['global', 'mid']].values:
    global2mid.at[from_, to_] += 1

In [281]:
for from_, to_ in kegg_pathways[['mid', 'local']].values:
    mid2local.at[from_, to_] += 1

In [288]:
for from_, to_ in kegg_pathways.join(pathways2genes, lsuffix='local', rsuffix='pathway', how='inner').dropna()[['local', 'tf']].values:
    for tf in to_:
        local2tf.at[from_, tf]+=1
        